# Lesson 2: Querying Knowledge Graphs with Cypher

### Import packages and set up Neo4

In [1]:
from dotenv import load_dotenv
import os

from langchain_neo4j import Neo4jGraph

In [2]:
load_dotenv()
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')


- Initialize a knowledge graph instance using LangChain's Neo4j integration

In [3]:
kg = Neo4jGraph(
    url = NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE
)

### Querying the movie knowledge graph 
- Match all nodes in the graph

In [4]:
cypher = """
MATCH (n)
RETURN count(n)
"""

In [5]:
result = kg.query(cypher)
result

[{'count(n)': 431}]

In [6]:
cypher = """
  MATCH (n) 
  RETURN count(n) AS numberOfNodes
  """

In [7]:
result = kg.query(cypher)
result

[{'numberOfNodes': 431}]

- Match only the `Products` nodes by specifying the node label

In [8]:
cypher = """
  MATCH (n:Product) 
  RETURN count(n) AS numberOfProducts
  """
kg.query(cypher)

[{'numberOfProducts': 10}]

- Match a single person by specifying the value of the `name` property on the `Person` node

In [9]:
cypher = """
  MATCH (c:Location {name: "Cambridge"}) 
  RETURN count(c) AS numberOfCambridge
  """
kg.query(cypher)

[{'numberOfCambridge': 1}]

### Cypher patterns with conditional matching

In [11]:
cypher = """
  MATCH (nineties:Movie) 
  WHERE nineties.released >= 1990 
    AND nineties.released < 2000 
  RETURN nineties.title
  """
kg.query(cypher)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: Movie)} {position: line: 2, column: 19, offset: 19} for query: '\n  MATCH (nineties:Movie) \n  WHERE nineties.released >= 1990 \n    AND nineties.released < 2000 \n  RETURN nineties.title\n  '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement

[]

### Pattern matching with multiple nodes

In [12]:
cypher = """
  MATCH (actor:Person)-[:ACTED_IN]->(movie:Movie) 
  RETURN actor.name, movie.title LIMIT 10
  """
kg.query(cypher)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: ACTED_IN)} {position: line: 2, column: 26, offset: 26} for query: '\n  MATCH (actor:Person)-[:ACTED_IN]->(movie:Movie) \n  RETURN actor.name, movie.title LIMIT 10\n  '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in

[]

In [13]:
cypher = """
  MATCH (tom:Person {name: "Tom Hanks"})-[:ACTED_IN]->(tomHanksMovies:Movie) 
  RETURN tom.name,tomHanksMovies.title
  """
kg.query(cypher)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: ACTED_IN)} {position: line: 2, column: 44, offset: 44} for query: '\n  MATCH (tom:Person {name: "Tom Hanks"})-[:ACTED_IN]->(tomHanksMovies:Movie) \n  RETURN tom.name,tomHanksMovies.title\n  '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when y

[]

In [14]:
cypher = """
  MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors) 
  RETURN coActors.name, m.title
  """
kg.query(cypher)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: ACTED_IN)} {position: line: 2, column: 43, offset: 43} for query: '\n  MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors) \n  RETURN coActors.name, m.title\n  '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or th

[]

### Delete data from the graph

In [15]:
cypher = """
MATCH (emil:Person {name:"Emil Eifrem"})-[actedIn:ACTED_IN]->(movie:Movie)
RETURN emil.name, movie.title
"""
kg.query(cypher)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: ACTED_IN)} {position: line: 2, column: 51, offset: 51} for query: '\nMATCH (emil:Person {name:"Emil Eifrem"})-[actedIn:ACTED_IN]->(movie:Movie)\nRETURN emil.name, movie.title\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this st

[]

In [16]:
cypher = """
MATCH (emil:Person {name:"Emil Eifrem"})-[actedIn:ACTED_IN]->(movie:Movie)
DELETE actedIn
"""
kg.query(cypher)

[]

### Adding data to the graph

In [17]:
cypher = """
CREATE (andreas:Person {name:"Andreas"})
RETURN andreas
"""

kg.query(cypher)

[{'andreas': {'name': 'Andreas'}}]

In [18]:
cypher = """
MATCH (andreas:Person {name:"Andreas"}), (emil:Person {name:"Emil Eifrem"})
MERGE (andreas)-[hasRelationship:WORKS_WITH]->(emil)
RETURN andreas, hasRelationship, emil
"""
kg.query(cypher)

[]